In [9]:
import re
import string

In [3]:
# load doc into memory
def load_doc(filename):
# open the file as read only
    file = open(filename, 'r' )
# read all text
    text = file.read()
# close the file
    file.close()
    return text

In [4]:
# load document
in_filename = '/home/ubuntu/Documents/plato_data.txt'
doc = load_doc(in_filename)
print(doc[:200])

﻿

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in w


In [5]:
# turn a doc into clean tokens
def clean_doc(doc):
    # replace ' -- ' with a space ' '
    doc = doc.replace( ' -- ' , ' ' )
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile( ' [%s] ' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub( '' , w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [10]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print( ' Total Tokens: %d ' % len(tokens))
print( ' Unique Tokens: %d ' % len(set(tokens)))

['book', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'the', 'thracian', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'which', 'was', 'a', 'new', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'but', 'that', 'of', 'the', 'thracians', 'was', 'if', 'not', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid', 'us', 'wait', 'for', 'the', 'servant', 'took', 'hold', 'of', 'me', 'by', 'the', 'cloak', 'and', 'polemarchus', 'desires', 'you', 'to'

In [13]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
  # select sequence of tokens
  seq = tokens[i-length:i]
  # convert into a line
  line = ' ' .join(seq)
  # store
  sequences.append(line)
print( ' Total Sequences: %d ' % len(sequences))

 Total Sequences: 100933 


In [16]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = ' \n ' .join(lines)
    file = open(filename, 'w' )
    file.write(data)
    file.close()

In [17]:
# save sequences to file
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

In [20]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r' )
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split( ' \n ' )

In [28]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

Using TensorFlow backend.


In [29]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r' )
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [33]:
# define the model
def define_model(vocab_size, seq_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 50, input_length=seq_length))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(100, activation= 'relu' ))
    
    model.add(Dense(vocab_size, activation= 'softmax' ))
    # compile network
    model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
    # summarize defined model
    model.summary()
    plot_model(model, to_file= 'model.png' , show_shapes=True)
    return model

In [34]:
# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split( ' \n ' )

In [38]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
# define model
model = define_model(vocab_size, seq_length)
# fit model
model.fit(X, y, batch_size=128, epochs=5)
# save the model to file
model.save( 'model.h5' )
# save the tokenizer
dump(tokenizer, open( 'tokenizer.pkl' , 'wb' ))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 50)            310700    
_________________________________________________________________
lstm_9 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 6214)              627614    
Total params: 1,089,214
Trainable params: 1,089,214
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
100933/100933 [==============================] - 163s 2ms/step - loss: 5.9205 - acc: 0.0757
Epoch 2/5
100933/100933 [=====

In [45]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [46]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r' )
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [47]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating= 'pre' )
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
            break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' ' .join(result)

In [48]:
# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split( ' \n ' )
seq_length = len(lines[0].split()) - 1
# load the model
model = load_model( 'model.h5' )
# load the tokenizer
tokenizer = load(open( 'tokenizer.pkl' , 'rb' ))

In [50]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + ' \n ' )
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

are injured be deteriorated in that which is the proper virtue of and that human virtue is to be then men who are injured are of necessity made that is the but can the musician by his art make men certainly or the horseman by his art make them bad and 
 
the                                                 
